# TM10007: Machine learning
## Week 4, lecture 1: Neural networks
#### Author: Karin A. van Garderen/ Muhammad Arif
For this exercise, we will learn to work with the basic neural network that is implemented in scikit-learn:
[MLPClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier)



## Preparing the datasset
We will be using the MNIST dataset, containing handwritten digits in 28 by 28 pixel images. In this case, each pixel is a feature and the labels are the numbers 0 through 9.
This part of the exercise was extended from [this example provided by scikit-learn](https://scikit-learn.org/stable/auto_examples/neural_networks/plot_mnist_filters.html#sphx-glr-auto-examples-neural-networks-plot-mnist-filters-py)

First, let's download the data, split it in a train and test set and show some examples.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


# Load data from https://www.openml.org/d/554
X, y = fetch_openml("mnist_784", version=1, return_X_y=True, as_frame=False)
X = X / 255.0

# Split data into train partition and test partition
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.7)

# Plot a few examples
fig, axes = plt.subplots(4, 4)
for sample, ax in zip(X_train[0:16], axes.ravel()):
    ax.matshow(sample.reshape(28, 28), cmap=plt.cm.gray, vmin=0, vmax=1)
    ax.set_xticks(())
    ax.set_yticks(())

plt.show()

## Training the network
We will be using the neural network classifier implemented in scikit-learn, called the Multi-layer Perceptron. This class takes care of all the complexities of optimizing the network, while allowing some modifications through hyperparameters. It can handle any number of inputs and outputs, and will adapt the number of connections accordingly.

### Network design
First of all, we can design the network architecture by assigning the number of hidden layers and the number of neurons (hidden features) in those layers. Simply input a tuple of as many layers as you want, with the number of neurons in each layer.

Let's start with a network that has no hidden layers at all. As the output is directly linked to the input, the network will have 784 x 10 = 7840 weights. We can still visualise those very easily, because they represent the weights of all the pixels in predicting each of the output digits.

In [ ]:
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier

## We set the learning rate relatively high so that we get our results quickly
mlp = MLPClassifier(
        hidden_layer_sizes=(),
        learning_rate_init=0.01
    )

mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

fig, axes = plt.subplots(2, 5)
# use global min / max to ensure all weights are shown on the same scale
vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()

print(f"Number of weights: {mlp.coefs_[0].size}")
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=0.5 * vmin, vmax=0.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()

- Can you recognize all the digits?
- What do you notice about the distribution of the weights?
- What does a very high value mean? Or a very low value?
- And what about the grey (in-between) values?

## Adding hidden layers
Now let's add some hidden layers and see what happens. The hidden layer can form intermediate features, in order to model more complex relations between the input and output. We can still plot the weights of the first hidden layer as images, but they are no longer directly linked to the 10 digits.

In [ ]:
print("Training a neural network with 40 hidden features.")

mlp = MLPClassifier(
        hidden_layer_sizes=(40,),
        learning_rate_init=0.01
    )

mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

# Plot the weights of the first layer again
fig, axes = plt.subplots(1, 5)
vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=0.5 * vmin, vmax=0.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()

print("Training a neural network with two layers of 40 hidden features.")

mlp = MLPClassifier(
        hidden_layer_sizes=(40,40),
        learning_rate_init=0.01
    )

mlp.fit(X_train, y_train)

print("Training set score: %f" % mlp.score(X_train, y_train))
print("Test set score: %f" % mlp.score(X_test, y_test))

# Plot the weights of the first layer again
fig, axes = plt.subplots(1, 5)
vmin, vmax = mlp.coefs_[0].min(), mlp.coefs_[0].max()
for coef, ax in zip(mlp.coefs_[0].T, axes.ravel()):
    ax.matshow(coef.reshape(28, 28), cmap=plt.cm.gray, vmin=0.5 * vmin, vmax=0.5 * vmax)
    ax.set_xticks(())
    ax.set_yticks(())
plt.show()


- How has the distribution of the weights changed?
- Can you still recognize digits in these images?
- Does adding this hidden layer also improve the performance?

## Learning rate
Next, let's have a look at the influence of the learning rate. As you know, the learning rate defines the size of the steps in each iteration of the gradient descent optimizer. A higher learning rate might get you to the optimal solution faster, but it might also cause noisy behavior and a worse final solution.

To investigate the convergence of the optimization, we can plot the loss at each step or 'epoch'. Let's do that for a variation of learning rates and see what happens.

In [ ]:
for learning_rate in [0.001, 0.01, 0.1, 0.5]:

    print(f"Training with learning rate {learning_rate}")
    ## To prevent very long waiting times, we set a maximum number of iterations
    mlp = MLPClassifier(
        hidden_layer_sizes=(40,),
        max_iter=100,
        alpha=1e-4,
        solver="sgd",
        random_state=1,
        learning_rate_init=learning_rate
    )


    mlp.fit(X_train, y_train)

    print("Training set score: %f" % mlp.score(X_train, y_train))
    print("Test set score: %f" % mlp.score(X_test, y_test))

    plt.plot(mlp.loss_curve_, label=f'lr = {learning_rate}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
plt.legend()
plt.show()


- From these results, can you tell what is the best learning rate for this problem?
- Do you expect this to be the same across different problems and networks?

## Generalization
From the loss curve it seems that our neural network can get a near perfect performance, but how will it perform on unseen data? By tracking the accuracy on the test set for each epoch, we can get see how the actual performance changes during training. By comparing the accuracy of training and test set, we can get a sense of the degree of overfitting.

In [ ]:
mlp = MLPClassifier(
    hidden_layer_sizes=(100),
    random_state=1,
    learning_rate_init=0.01
)

# The MLPClassifier doesn't automatically log the accuracies during training,
# so we will step through the optimization manually and log the performance
# at each step.
train_scores = []
test_scores = []
n = 40

for i in range(0,n):
    mlp.partial_fit(X_train, y_train, np.unique(y_train))
    train_scores.append(mlp.score(X_train, y_train))
    test_scores.append(mlp.score(X_test, y_test))

plt.plot(train_scores, label='Training accuracy')
plt.plot(test_scores, label='Test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

- Would you consider this classifier overfitted? Why (not)?

## Regularization
To illustrate the mechanisms of overfitting and regularization in neural networks, we will use a dataset with fewer samples and a higher dimensionality. For this we will use the text dataset that was introduced in the previous exercise, this time with 2000 extracted features.

The `MLPClassifier` implemented in Scikit-learn allows for L2 regularization using the `alpha` parameter. This encourages the weights to become smaller, in theory limiting the potential for overfitting. We will try this with some extreme values of `alpha` and track the accuracy and loss.


In [ ]:
### Code to load and transform the dataset

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from time import time

def load_text_dataset(N_features=100):
    '''
    Load dataset for classifying text documents by topic.
    '''
    categories = [
        'alt.atheism',
        'talk.religion.misc'
    ]

    remove = ('headers', 'footers', 'quotes')

    print("Loading 20 newsgroups dataset for categories:")
    print(categories if categories else "all")

    data_train = fetch_20newsgroups(subset='train', categories=categories,
                                    shuffle=True, random_state=42,
                                    remove=remove)

    data_test = fetch_20newsgroups(subset='test', categories=categories,
                                   shuffle=True, random_state=42,
                                   remove=remove)
    print('data loaded')

    # order of labels in `target_names` can be different from `categories`
    target_names = data_train.target_names


    def size_mb(docs):
        return sum(len(s.encode('utf-8')) for s in docs) / 1e6


    data_train_size_mb = size_mb(data_train.data)
    data_test_size_mb = size_mb(data_test.data)

    print("%d documents - %0.3fMB (training set)" % (
        len(data_train.data), data_train_size_mb))
    print("%d documents - %0.3fMB (test set)" % (
        len(data_test.data), data_test_size_mb))
    print("%d categories" % len(target_names))
    print()

    # split a training set and a test set
    y_train, y_test = data_train.target, data_test.target

    print("Extracting features from the training data using a sparse vectorizer")
    t0 = time()
    use_hashing = False
    if use_hashing:
        vectorizer = HashingVectorizer(stop_words='english', alternate_sign=False,
                                       n_features=2 ** 16)
        X_train = vectorizer.transform(data_train.data)
    else:
        vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                     stop_words='english')
        X_train = vectorizer.fit_transform(data_train.data)
    duration = time() - t0
    print("done in %fs at %0.3fMB/s" % (duration, data_train_size_mb / duration))
    print("n_samples: %d, n_features: %d" % X_train.shape)
    print()

    print("Extracting features from the test data using the same vectorizer")
    t0 = time()
    X_test = vectorizer.transform(data_test.data)
    duration = time() - t0
    print("done in %fs at %0.3fMB/s" % (duration, data_test_size_mb / duration))
    print("n_samples: %d, n_features: %d" % X_test.shape)
    print()

    # mapping from integer feature name to original token string
    if use_hashing:
        feature_names = None
    else:
        #feature_names = vectorizer.get_feature_names()
        feature_names = vectorizer.get_feature_names_out()

    if N_features < X_train.shape[1]:
        print("Extracting %d best features by a chi-squared test" %
              N_features)
        t0 = time()
        ch2 = SelectKBest(chi2, k=N_features)
        X_train = ch2.fit_transform(X_train, y_train)
        X_test = ch2.transform(X_test)
        # if feature_names:
        if len(feature_names) != 0:
            # keep selected feature names
            feature_names = [feature_names[i] for i
                             in ch2.get_support(indices=True)]
        print("done in %fs" % (time() - t0))
        print()

    if feature_names:
        feature_names = np.asarray(feature_names)

    return X_train, X_test, y_train, y_test

In [ ]:
X_train_text, X_test_text, y_train_text, y_test_text = load_text_dataset(N_features=2000)

# Prepare the plots
fig, axes = plt.subplots(1, 3, figsize=(20,10))
# Number of epochs
n=200

# Train neural networks with different values
# of the regularization parameter alpha
for alpha in [0, 0.1, 1]:
    print(f'Training with alpha = {alpha}')
    mlp = MLPClassifier(
            hidden_layer_sizes=(100),
            alpha=alpha,
            random_state=1,
            learning_rate_init=0.001
        )

    train_scores = []
    test_scores = []

    # Track the train and test performances for every training step
    for i in range(0,n):
        mlp.partial_fit(X_train_text, y_train_text, np.unique(y_train_text))
        train_scores.append(mlp.score(X_train_text, y_train_text))
        test_scores.append(mlp.score(X_test_text, y_test_text))

    axes[0].plot(test_scores, label=f'Alpha = {alpha}')
    axes[1].plot(train_scores, label=f'Alpha = {alpha}')
    axes[2].plot(mlp.loss_curve_, label=f'Alpha = {alpha}')

axes[0].set_xlabel('Epoch')
axes[1].set_xlabel('Epoch')
axes[2].set_xlabel('Epoch')
axes[0].set_ylabel('Test accuracy')
axes[1].set_ylabel('Train accuracy')
axes[2].set_ylabel('Train loss')
axes[0].legend()
axes[1].legend()
axes[2].legend()
plt.show()

- Does regularization improve the performance for this classifier?
- How would you find the optimal value for alpha?
- Can you think of other ways to reduce overfitting?

## Early stopping
When training neural networks we sometimes notice that overfitting gets worse as we keep training, and the performance on the test set starts decreasing. A simple way to improve performance is to look back at the loss curve and select the parameters where the performance on a validation set is at the highest point.

The MLPClassifier can implement such a scheme, called 'Early stopping', and will automatically keep a part of the training set separate to track the validation loss. Let's see if this will improve performance for the network we were training before

In [ ]:
print("Without early stopping.")

mlp = MLPClassifier(
        hidden_layer_sizes=(100),
        alpha=0,
        random_state=1,
        learning_rate_init=0.001,
        early_stopping=False
    )

mlp.fit(X_train_text, y_train_text)

print("Training set score: %f" % mlp.score(X_train_text, y_train_text))
print("Test set score: %f" % mlp.score(X_test_text, y_test_text))

print("With early stopping.")

mlp = MLPClassifier(
        hidden_layer_sizes=(100),
        alpha=0,
        random_state=1,
        learning_rate_init=0.001,
        early_stopping=True
    )

mlp.fit(X_train_text, y_train_text)

print("Training set score: %f" % mlp.score(X_train_text, y_train_text))
print("Test set score: %f" % mlp.score(X_test_text, y_test_text))


- Did the performance improve as much as you had expected?
- Can you think of a reason not to use early stopping?
- Why do we need a separate validation set? What would happen if we used the test set for early stopping?

## Conclusion
The MLPClassifier offers a simple implementation of a neural network for classification. Have a look at the documentation to see all the hyperparameters, because
we did not cover them all.

Remember that the optimal network architecture and hyperparameters may vary a lot between datasets. When designing your network, it is worthwhile to log the loss curve, both for the training and validation/test set, because it will give some insight in the behavior of the network.
Questions to consider:
- Is the optimization converging to slowly, or is there a lot of noise?
- Is the network overfitting? If so, you could try to reduce the network size or apply regularization.

Remember though: we would expect some difference between the train and test performance, and this is not always a reason to be concerned!